## Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Importing Excel File

In [2]:
df = pd.read_excel('data/Shiller_Ch26_data_modified.xlsx')

In [3]:
print(df.columns)

Index(['Date', 'S&P', 'Dividends', 'Earnings', 'One-Year', 'Long', 'Consumer',
       'Real One-Year Interest Rate', 'Real Per Capita Consumption',
       'Unnamed: 9', 'RealP', 'Present', 'Present.1', 'Present.2', 'RealD',
       'Real Returns S&P', 'Unnamed: 16', 'RealE', 'Price', 'Ten-Year',
       'Price.1'],
      dtype='object')


In [4]:
## Creating a dataframe only with interest rates, consumptioin, and stock returns
df = df.loc[:,['Date','Real One-Year Interest Rate','Real Per Capita Consumption','Real Returns S&P']]

In [5]:
df.dropna(inplace=True)

In [6]:
## Obtaining Consumption Growth 
df['Consumption Growth'] = df['Real Per Capita Consumption'].pct_change() + 1

In [7]:
df.dropna(inplace=True)

In [8]:
df.head(5)

,Date,Real One-Year Interest Rate,Real Per Capita Consumption,Real Returns S&P,Consumption Growth
19,1890,1.028390,2674.739801,-0.082419,0.979195
20,1891,1.128512,2802.827499,0.260760,1.047888
21,1892,0.964169,2877.010456,-0.015044,1.026467
22,1893,1.250995,2834.846162,-0.060927,0.985344
23,1894,1.078121,2698.707529,0.077294,0.951977


## Stating the Problem 

We consider an economy with **two Markov-chain states**:

Normal (state 1)

Recession (state 2)

The probability of transitioning from normal to recession is $ \pi_{12} = 0.08 $.

The probability of transitioning from recession back to normal is $ \pi_{21} $ (to be calibrated).

We denote the (gross) consumption growth in each state by $ G_1 $ and $ G_2 $, respectively, with $ G_2 = 0.97 $ (i.e., a 3% drop in recessions).

**Long-Run Probabilities**

The long-run (stationary) distribution is given by
$ \Pi_1 = \frac{\pi_{21}}{\pi_{12} + \pi_{21}}, \quad \Pi_2 = 1 - \Pi_1. $

We calibrate $ \pi_{21} $ so that recessions occur every 10 years on average, i.e., $ \Pi_2 = 0.10 $.

**Matching Mean Consumption Growth**

Empirically, let $ \bar{G} $ be the observed mean of the gross consumption growth. We match this average in the model via

$ \bar{G} = \Pi_1 , G_1$

$Pi_2 , G_2, $ solving for $ G_1 $.

**Euler Equation**

With time discount factor $ \beta = 0.99 $ and risk aversion $ \gamma = 5 $, the investor’s Euler equation in each state $ i \in {1,2} $ is

$ 1 = \beta \sum_{j=1}^2 \pi_{ij} , (G_j)^{-\gamma} , R_j. $ 

Solving these two simultaneous equations yields the state-dependent stock returns $ R_1 $ and $ R_2 $.

**Risk-Free Rates**

The risk-free rate in each state is

$ R_{f,i} = \frac{1}{ \beta \sum_{j=1}^2 \pi_{ij} , (G_j)^{-\gamma} }. $

**Mean Equity Premium**

We define the mean equity return as the stationary average of the stock returns, 

Mean equity premium = $\left[\Pi_1R_1+\Pi_2R_2\right]-\left[\Pi_1R_{f,1}+\Pi_2R_{f,2}\right]$

By carrying out these steps (finding $ \pi_{21} $, calibrating $ G_1 $, and then solving the Euler equations), we obtain state-dependent returns and the mean equity premium in the two-state setup.

In [10]:
## Computing empirical mean consumption growth from the data
mean_consumption_growth = df['Consumption Growth'].mean()

## Given parameters
beta = 0.99
gamma = 5
G2 = 0.97
pi_12 = 0.08  # Probability expansion -> recession

## We know recessions occur every 10 years on average, so Pi_2 = 0.1 and Pi_1 = 0.9
Pi_1 = 0.9
Pi_2 = 0.1

## Solving for pi_21
    # Pi_1 = pi_21 / (pi_12 + pi_21)
    # 0.9 = pi_21/(0.08 + pi_21)
    # 0.9 * (0.08 + pi_21) = pi_21
    # 0.072 + 0.9*pi_21 = pi_21
    # 0.072 = 0.1*pi_21 => pi_21 = 0.72
pi_21 = 0.72
pi_11 = 1 - pi_12  ## = 0.92
pi_22 = 1 - pi_21  ## = 0.28

## Solving for G_1 so that mean consumption growth matches data:
## mean_consumption_growth = Pi_1 * G_1 + Pi_2 * G_2
G1 = (mean_consumption_growth - Pi_2 * G2) / Pi_1

## Now we have everything to solve the Euler equations for R_1 and R_2
    # 1 = beta[ pi_11*(G1)^(-gamma)*R_1 + pi_12*(G2)^(-gamma)*R_2 ]  (state 1)
    # 1 = beta[ pi_21*(G1)^(-gamma)*R_1 + pi_22*(G2)^(-gamma)*R_2 ]  (state 2)

G1_neg_gamma = G1**(-gamma)
G2_neg_gamma = G2**(-gamma)

A = np.array([
    [beta*pi_11*G1_neg_gamma, beta*pi_12*G2_neg_gamma],
    [beta*pi_21*G1_neg_gamma, beta*pi_22*G2_neg_gamma]
])
b = np.array([1, 1])

## Solving for R_1 and R_2
R1, R2 = np.linalg.solve(A, b)

## Computing the risk-free rates in each state
## R_{f,i} = 1 / [ beta * (pi_{i1}(G_1)^{-gamma} + pi_{i2}(G_2)^{-gamma}) ]

Rf_1 = 1.0 / (beta * (pi_11*G1_neg_gamma + pi_12*G2_neg_gamma))
Rf_2 = 1.0 / (beta * (pi_21*G1_neg_gamma + pi_22*G2_neg_gamma))

## Computing the mean equity premium
## Mean(R_equity) = Pi_1*R_1 + Pi_2*R_2
## Mean(Rf) = Pi_1*Rf_1 + Pi_2*Rf_2   -> Risk-free
mean_equity_premium = (Pi_1*R1 + Pi_2*R2) - (Pi_1*Rf_1 + Pi_2*Rf_2)

## Printing results
print("Calibrated pi_21:", pi_21)
print("Calibrated G_1:", G1)
print("State-dependent stock returns: R_1 = {}, R_2 = {}".format(R1, R2))
print("State-dependent risk-free rates: Rf_1 = {}, Rf_2 = {}".format(Rf_1, Rf_2))
print(f"Mean Equity Premium:{mean_equity_premium:.4f}")

Calibrated pi_21: 0.72
Calibrated G_1: 1.0264162109065862
State-dependent stock returns: R_1 = 1.1507535360796606, R_2 = 0.8674081067676769
State-dependent risk-free rates: Rf_1 = 1.1214471982342826, Rf_2 = 1.0543209929927526
Mean Equity Premium:0.0077


Thus, the mean equity premium under two states is approximately 0.0077.

## What about Three States?

We extend the 2-state model by adding a disaster state (state 3). Label the states:

Normal (state 1)

Recession (state 2)

Disaster (state 3)

**Consumption growth** in each state is denoted by
$ G_1, \quad G_2 = 0.97, \quad G_3 = 0.70 $ (so a 30% drop in disaster).

**Transition Matrix**

We have a Markov chain with transition probabilities
$ \pi_{ij} = P(\text{state } j \mid \text{state } i), $ forming a $3\times 3$ matrix 
$ P = \begin{pmatrix}
\pi_{11} & \pi_{12} & \pi_{13} \\
\pi_{21} & \pi_{22} & \pi_{23} \\
\pi_{31} & \pi_{32} & \pi_{33}
\end{pmatrix}. $
 

Typically,

From **normal** (state 1): a small probability $ \pi_{13} $ of moving to disaster;

From **disaster** (state 3): the economy returns to normal next period ($ \pi_{31} = 1 $);

From **recession** (state 2): no direct transition to disaster ($ \pi_{23} = 0 $);

The rest of the probabilities sum to 1 accordingly.

**Stationary Distribution**

The long-run distribution $ \Pi = (\Pi_1,, \Pi_2,, \Pi_3) $ solves
$ \Pi = \Pi , P \quad\text{and}\quad \Pi_1 + \Pi_2 + \Pi_3 = 1. $ 

Numerically, one can find $ \Pi $ by computing the eigenvector of $ P^\top $ corresponding to the eigenvalue 1.

**Calibrating $G_1$**

Given an empirical mean consumption growth $ \bar{G} $, we match it in the model via
$ \bar{G} = \Pi_1 , G_1$

$\Pi_2 , G_2$
$\Pi_3 , G_3, $ solving for $ G_1 $.

**Euler Equations**

With time discount factor $ \beta = 0.99 $ and risk aversion $ \gamma = 5 $, the Euler equation for each state $ i \in {1,2,3} $ is
$ 1 = \beta \sum_{j=1}^{3} \pi_{ij},(G_j)^{-\gamma},R_j, $ giving a system of three equations in the unknowns $ R_1, R_2, R_3 $ (the state-dependent stock returns).

**Risk-Free Rates**

The risk-free rate in each state is computed by pricing a risk-free asset that pays 1 in every next-period state: $ R_{f,i} = \frac{1}{ \beta \sum_{j=1}^{3} \pi_{ij},(G_j)^{-\gamma} }. $

**Mean Equity Premium**

$\left(\Pi_1R_1+\Pi_2R_2+\Pi_3R_3\right)-\left(\Pi_1R_{f,1}+\Pi_2R_{f,2}+\Pi_3R_{f,3}\right)$

By performing these steps—solving for the stationary distribution, calibrating $ G_1 $, then solving the three Euler equations—we obtain the state-dependent returns, risk-free rates, and the mean equity premium in this three-state economy with rare disasters.

In [11]:
## Computing mean consumption growth 
mean_consumption_growth = df["Consumption Growth"].mean()

## Given Parameters
beta = 0.99
gamma = 5

## Growth rates
G2 = 0.97
G3 = 0.70  ## Disaster: consumption drops by 30%

## Transition probabilities (3-state)
## From previous scenario, adjusted for disaster (making the P matrix):
pi_11 = 0.91
pi_12 = 0.07
pi_13 = 0.02

pi_21 = 0.72
pi_22 = 0.28
pi_23 = 0.00

pi_31 = 1.00
pi_32 = 0.00
pi_33 = 0.00

P = np.array([
    [pi_11, pi_12, pi_13],
    [pi_21, pi_22, pi_23],
    [pi_31, pi_32, pi_33]
])

## Solving ΠP = Π with Π_1+Π_2+Π_3=1
    ## This can be done by finding the eigenvector of P' corresponding to eigenvalue 1 (what I will do)
    ## or by solving (P' - I)*Pi = 0 with sum(Pi)=1.

eigs, vecs = np.linalg.eig(P.T)

## Finding eigenvector for eigenvalue ~1
idx = np.argmin(np.abs(eigs - 1))
stationary = np.real(vecs[:, idx])

## Normalizing
stationary = stationary / stationary.sum()
Pi_1, Pi_2, Pi_3 = stationary

## Solving for G_1 so mean consumption growth matches data:
    ## mean_consumption_growth = Pi_1*G_1 + Pi_2*G_2 + Pi_3*G_3
G1 = (mean_consumption_growth - Pi_2*G2 - Pi_3*G3) / Pi_1

## Computing (G_j)^{-gamma}:
G1_neg_gamma = G1**(-gamma)
G2_neg_gamma = G2**(-gamma)
G3_neg_gamma = G3**(-gamma)

## Euler equations:
# For state 1:
    ## 1 = beta[ pi_11*G1^-gamma * R_1 + pi_12*G2^-gamma * R_2 + pi_13*G3^-gamma * R_3 ]
# State 2:
    ## 1 = beta[ pi_21*G1^-gamma * R_1 + pi_22*G2^-gamma * R_2 + pi_23*G3^-gamma * R_3 ]
# State 3:
    ## 1 = beta[ pi_31*G1^-gamma * R_1 + pi_32*G2^-gamma * R_2 + pi_33*G3^-gamma * R_3 ]

A = np.array([
    [beta*pi_11*G1_neg_gamma, beta*pi_12*G2_neg_gamma, beta*pi_13*G3_neg_gamma],
    [beta*pi_21*G1_neg_gamma, beta*pi_22*G2_neg_gamma, beta*pi_23*G3_neg_gamma],
    [beta*pi_31*G1_neg_gamma, beta*pi_32*G2_neg_gamma, beta*pi_33*G3_neg_gamma]
])
b = np.array([1.0, 1.0, 1.0])

## Solving for R_1, R_2, R_3
R1, R2, R3 = np.linalg.solve(A, b)

## Computing risk-free rates
## Rf_i = 1 / [beta * ( pi_i1*G1^-gamma + pi_i2*G2^-gamma + pi_i3*G3^-gamma )]
Rf_1 = 1.0 / (beta * (pi_11*G1_neg_gamma + pi_12*G2_neg_gamma + pi_13*G3_neg_gamma))
Rf_2 = 1.0 / (beta * (pi_21*G1_neg_gamma + pi_22*G2_neg_gamma + pi_23*G3_neg_gamma))
Rf_3 = 1.0 / (beta * (pi_31*G1_neg_gamma + pi_32*G2_neg_gamma + pi_33*G3_neg_gamma))

## Computing mean equity premium
mean_equity_return = Pi_1*R1 + Pi_2*R2 + Pi_3*R3
mean_rf = Pi_1*Rf_1 + Pi_2*Rf_2 + Pi_3*Rf_3
mean_equity_premium = mean_equity_return - mean_rf

## Printing values
print("Stationary distribution: Pi_1 = {}, Pi_2 = {}, Pi_3 = {}".format(Pi_1, Pi_2, Pi_3))
print("Calibrated G_1:", G1)
print("State-dependent stock returns: R_1 = {}, R_2 = {}, R_3 = {}".format(R1, R2, R3))
print("State-dependent risk-free rates: Rf_1 = {}, Rf_2 = {}, Rf_3 = {}".format(Rf_1, Rf_2, Rf_3))
print(f"Mean Equity Premium:{mean_equity_premium:.4f}")

Stationary distribution: Pi_1 = 0.8950770760815515, Pi_2 = 0.08702138239681749, Pi_3 = 0.017901541521631037
Calibrated G_1: 1.0321265000665722
State-dependent stock returns: R_1 = 1.1831217801631224, R_2 = 0.8674081067676767, R_3 = 0.16976767676767648
State-dependent risk-free rates: Rf_1 = 1.0334210019291827, Rf_2 = 1.0736983712789852, Rf_3 = 1.1831217801631224
Mean Equity Premium:0.0979


Thus, the mean equity premium under three states is approximately 0.0979.